In [1]:
from hurricanes.models import rtofs
import numpy as np
import pandas as pd
import xarray as xr

In [2]:
extent = [-71, -62, 15, 22]

In [3]:
with rtofs() as ds:
    # Save rtofs lon and lat as variables to speed up indexing calculation
    grid_lons = ds.lon.values[0,:]
    grid_lats = ds.lat.values[:,0]
    grid_x = ds.x.values
    grid_y = ds.y.values
    
    # Find x, y indexes of the area we want to subset
    lons_ind = np.interp(extent[:2], grid_lons, grid_x)
    lats_ind = np.interp(extent[2:], grid_lats, grid_y)

    # Use np.floor on the 1st index and np.ceil on the 2nd index of each slice 
    # in order to widen the area of the extent slightly.
    extent_ind = [
        np.floor(lons_ind[0]).astype(int),
        np.ceil(lons_ind[1]).astype(int),
        np.floor(lats_ind[0]).astype(int),
        np.ceil(lats_ind[1]).astype(int)
        ]

In [4]:
url = "https://tds.marine.rutgers.edu/thredds/dodsC/cool/rtofs/rtofs_us_east_scraped"
rds = xr.open_dataset(url, chunks={"x": 371, "y": 342, "time": 20}).drop_vars(['u', 'v'])
rds

<xarray.Dataset>
Dimensions:      (depth: 40, y: 1710, x: 742, time: 4475)
Coordinates:
  * depth        (depth) float32 0.0 2.0 4.0 6.0 ... 2.5e+03 3e+03 4e+03 5e+03
  * y            (y) int32 1 2 3 4 5 6 7 ... 1704 1705 1706 1707 1708 1709 1710
  * x            (x) int32 1 2 3 4 5 6 7 8 9 ... 735 736 737 738 739 740 741 742
  * time         (time) datetime64[ns] 2019-07-17T06:00:00 ... 2022-08-25
Data variables:
    date         (time) float64 dask.array<chunksize=(20,), meta=np.ndarray>
    lat          (y, x) float32 dask.array<chunksize=(342, 371), meta=np.ndarray>
    lon          (y, x) float32 dask.array<chunksize=(342, 371), meta=np.ndarray>
    temperature  (time, depth, y, x) float32 dask.array<chunksize=(20, 40, 342, 371), meta=np.ndarray>
    salinity     (time, depth, y, x) float32 dask.array<chunksize=(20, 40, 342, 371), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.0
    title:        HYCOM ATLb2.00
    institution:  National Centers for Environmental Prediction
    source:       ftp://ftp.ncei.noaa.gov/pub/data/nccf/com/rtofs/prod/
    experiment:   92.8
    history:      archv2ncdf3z
    region:       Atlantic/US East Coast
    comment:      This dataset contains the first 4 full volume forecasts (f0...

In [7]:
# Use .isel selector on x/y since we know indexes that we want to slice
ds_slice = rds.isel(x=slice(extent_ind[0], extent_ind[1]), y=slice(extent_ind[2], extent_ind[3]))
ds_slice

<xarray.Dataset>
Dimensions:      (depth: 40, y: 94, x: 113, time: 4475)
Coordinates:
  * depth        (depth) float32 0.0 2.0 4.0 6.0 ... 2.5e+03 3e+03 4e+03 5e+03
  * y            (y) int32 191 192 193 194 195 196 ... 279 280 281 282 283 284
  * x            (x) int32 364 365 366 367 368 369 ... 471 472 473 474 475 476
  * time         (time) datetime64[ns] 2019-07-17T06:00:00 ... 2022-08-25
Data variables:
    date         (time) float64 dask.array<chunksize=(20,), meta=np.ndarray>
    lat          (y, x) float32 dask.array<chunksize=(94, 8), meta=np.ndarray>
    lon          (y, x) float32 dask.array<chunksize=(94, 8), meta=np.ndarray>
    temperature  (time, depth, y, x) float32 dask.array<chunksize=(20, 40, 94, 8), meta=np.ndarray>
    salinity     (time, depth, y, x) float32 dask.array<chunksize=(20, 40, 94, 8), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.0
    title:        HYCOM ATLb2.00
    institution:  National Centers for Environmental Prediction
    source:       ftp://ftp.ncei.noaa.gov/pub/data/nccf/com/rtofs/prod/
    experiment:   92.8
    history:      archv2ncdf3z
    region:       Atlantic/US East Coast
    comment:      This dataset contains the first 4 full volume forecasts (f0...

In [8]:
trange = pd.date_range(pd.Timestamp(2019, 7, 17, 12, 0, 0), pd.Timestamp(2021, 3, 4, 12, 0, 0), freq='1D')
trange

DatetimeIndex(['2019-07-17 12:00:00', '2019-07-18 12:00:00',
               '2019-07-19 12:00:00', '2019-07-20 12:00:00',
               '2019-07-21 12:00:00', '2019-07-22 12:00:00',
               '2019-07-23 12:00:00', '2019-07-24 12:00:00',
               '2019-07-25 12:00:00', '2019-07-26 12:00:00',
               ...
               '2021-02-23 12:00:00', '2021-02-24 12:00:00',
               '2021-02-25 12:00:00', '2021-02-26 12:00:00',
               '2021-02-27 12:00:00', '2021-02-28 12:00:00',
               '2021-03-01 12:00:00', '2021-03-02 12:00:00',
               '2021-03-03 12:00:00', '2021-03-04 12:00:00'],
              dtype='datetime64[ns]', length=597, freq='D')

In [9]:
ds_slice_trange = ds_slice.sel(time=trange, method='nearest')
ds_slice_trange

<xarray.Dataset>
Dimensions:      (depth: 40, y: 94, x: 113, time: 597)
Coordinates:
  * depth        (depth) float32 0.0 2.0 4.0 6.0 ... 2.5e+03 3e+03 4e+03 5e+03
  * y            (y) int32 191 192 193 194 195 196 ... 279 280 281 282 283 284
  * x            (x) int32 364 365 366 367 368 369 ... 471 472 473 474 475 476
  * time         (time) datetime64[ns] 2019-07-17T12:00:00 ... 2021-03-04T12:...
Data variables:
    date         (time) float64 dask.array<chunksize=(5,), meta=np.ndarray>
    lat          (y, x) float32 dask.array<chunksize=(94, 8), meta=np.ndarray>
    lon          (y, x) float32 dask.array<chunksize=(94, 8), meta=np.ndarray>
    temperature  (time, depth, y, x) float32 dask.array<chunksize=(5, 40, 94, 8), meta=np.ndarray>
    salinity     (time, depth, y, x) float32 dask.array<chunksize=(5, 40, 94, 8), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.0
    title:        HYCOM ATLb2.00
    institution:  National Centers for Environmental Prediction
    source:       ftp://ftp.ncei.noaa.gov/pub/data/nccf/com/rtofs/prod/
    experiment:   92.8
    history:      archv2ncdf3z
    region:       Atlantic/US East Coast
    comment:      This dataset contains the first 4 full volume forecasts (f0...

In [10]:
from dask.diagnostics import ProgressBar

# or distributed.progress when using the distributed scheduler
delayed_obj = ds_slice_trange.to_netcdf("rtofs_201907_20210304.nc", compute=False)

with ProgressBar():
    results = delayed_obj.compute()

[########################################] | 100% Completed |  1hr 27min  5.7s


In [11]:
rds2 = xr.open_dataset(url).drop_vars(['u', 'v'])
rds2

<xarray.Dataset>
Dimensions:      (depth: 40, y: 1710, x: 742, time: 4471)
Coordinates:
  * depth        (depth) float32 0.0 2.0 4.0 6.0 ... 2.5e+03 3e+03 4e+03 5e+03
  * y            (y) int32 1 2 3 4 5 6 7 ... 1704 1705 1706 1707 1708 1709 1710
  * x            (x) int32 1 2 3 4 5 6 7 8 9 ... 735 736 737 738 739 740 741 742
  * time         (time) datetime64[ns] 2019-07-17T06:00:00 ... 2022-08-24
Data variables:
    date         (time) float64 ...
    lat          (y, x) float32 ...
    lon          (y, x) float32 ...
    temperature  (time, depth, y, x) float32 ...
    salinity     (time, depth, y, x) float32 ...
Attributes:
    Conventions:  CF-1.0
    title:        HYCOM ATLb2.00
    institution:  National Centers for Environmental Prediction
    source:       ftp://ftp.ncei.noaa.gov/pub/data/nccf/com/rtofs/prod/
    experiment:   92.8
    history:      archv2ncdf3z
    region:       Atlantic/US East Coast
    comment:      This dataset contains the first 4 full volume forecasts (f0...

In [12]:
# Use .isel selector on x/y since we know indexes that we want to slice
ds2_slice = rds2.isel(x=slice(extent_ind[0], extent_ind[1]), y=slice(extent_ind[2], extent_ind[3]))
ds2_slice

<xarray.Dataset>
Dimensions:      (depth: 40, y: 94, x: 113, time: 4471)
Coordinates:
  * depth        (depth) float32 0.0 2.0 4.0 6.0 ... 2.5e+03 3e+03 4e+03 5e+03
  * y            (y) int32 191 192 193 194 195 196 ... 279 280 281 282 283 284
  * x            (x) int32 364 365 366 367 368 369 ... 471 472 473 474 475 476
  * time         (time) datetime64[ns] 2019-07-17T06:00:00 ... 2022-08-24
Data variables:
    date         (time) float64 2.019e+07 2.019e+07 ... 2.022e+07 2.022e+07
    lat          (y, x) float32 ...
    lon          (y, x) float32 ...
    temperature  (time, depth, y, x) float32 ...
    salinity     (time, depth, y, x) float32 ...
Attributes:
    Conventions:  CF-1.0
    title:        HYCOM ATLb2.00
    institution:  National Centers for Environmental Prediction
    source:       ftp://ftp.ncei.noaa.gov/pub/data/nccf/com/rtofs/prod/
    experiment:   92.8
    history:      archv2ncdf3z
    region:       Atlantic/US East Coast
    comment:      This dataset contains the first 4 full volume forecasts (f0...

In [13]:
ds2_slice_trange = ds2_slice.sel(time=trange, method='nearest')
ds2_slice_trange

<xarray.Dataset>
Dimensions:      (depth: 40, y: 94, x: 113, time: 1134)
Coordinates:
  * depth        (depth) float32 0.0 2.0 4.0 6.0 ... 2.5e+03 3e+03 4e+03 5e+03
  * y            (y) int32 191 192 193 194 195 196 ... 279 280 281 282 283 284
  * x            (x) int32 364 365 366 367 368 369 ... 471 472 473 474 475 476
  * time         (time) datetime64[ns] 2019-07-17T12:00:00 ... 2022-08-23T12:...
Data variables:
    date         (time) float64 2.019e+07 2.019e+07 ... 2.022e+07 2.022e+07
    lat          (y, x) float32 15.02 15.02 15.02 15.02 ... 22.07 22.07 22.07
    lon          (y, x) float32 -70.96 -70.88 -70.8 ... -62.16 -62.08 -62.0
    temperature  (time, depth, y, x) float32 ...
    salinity     (time, depth, y, x) float32 ...
Attributes:
    Conventions:  CF-1.0
    title:        HYCOM ATLb2.00
    institution:  National Centers for Environmental Prediction
    source:       ftp://ftp.ncei.noaa.gov/pub/data/nccf/com/rtofs/prod/
    experiment:   92.8
    history:      archv2ncdf3z
    region:       Atlantic/US East Coast
    comment:      This dataset contains the first 4 full volume forecasts (f0...

In [15]:
ds2_slice_trange_chunked = ds2_slice_trange.chunk({'time': 100})
ds2_slice_trange_chunked

<xarray.Dataset>
Dimensions:      (depth: 40, y: 94, x: 113, time: 1134)
Coordinates:
  * depth        (depth) float32 0.0 2.0 4.0 6.0 ... 2.5e+03 3e+03 4e+03 5e+03
  * y            (y) int32 191 192 193 194 195 196 ... 279 280 281 282 283 284
  * x            (x) int32 364 365 366 367 368 369 ... 471 472 473 474 475 476
  * time         (time) datetime64[ns] 2019-07-17T12:00:00 ... 2022-08-23T12:...
Data variables:
    date         (time) float64 dask.array<chunksize=(100,), meta=np.ndarray>
    lat          (y, x) float32 dask.array<chunksize=(94, 113), meta=np.ndarray>
    lon          (y, x) float32 dask.array<chunksize=(94, 113), meta=np.ndarray>
    temperature  (time, depth, y, x) float32 dask.array<chunksize=(100, 40, 94, 113), meta=np.ndarray>
    salinity     (time, depth, y, x) float32 dask.array<chunksize=(100, 40, 94, 113), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.0
    title:        HYCOM ATLb2.00
    institution:  National Centers for Environmental Prediction
    source:       ftp://ftp.ncei.noaa.gov/pub/data/nccf/com/rtofs/prod/
    experiment:   92.8
    history:      archv2ncdf3z
    region:       Atlantic/US East Coast
    comment:      This dataset contains the first 4 full volume forecasts (f0...

In [ ]:
from dask.diagnostics import ProgressBar

# or distributed.progress when using the distributed scheduler
delayed_obj = ds_slice_trange.to_netcdf("rtofs_201907_20220823.nc", compute=False)

with ProgressBar():
    results = delayed_obj.compute()